In [1]:
cd ..

/home/toor/study/mfti nlp/prune_labse_en_ru


/home/toor/miniconda3/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
import logging

from src.config import Config
from src.datamodule import SentenceDM
from src.lightning_module import PruneModule
from src.pruning_module import prune_model

In [3]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

os.environ["TOKENIZERS_PARALLELISM"] = "true"

def msize(m):
    return sum(p.numel() for p in m.parameters())

In [4]:
config = Config.from_yaml('config/config.yml')

datamodule = SentenceDM(config.data_config)
model = PruneModule(config)

print(f'Режим отладки: {config.data_config.debug}')
print(config.pruning)

Режим отладки: True
pruning_ratio=0.5 global_pruning=False iterative_steps=1 save_model=True


In [5]:
prune_model(model, datamodule)

Warning! No positional inputs found for a module, assuming batch size is 1.


INFO:root:Model complexity: 176762.91456 MMAC, 128.345088 M params
100%|███████████████████████████████████████████| 63/63 [00:05<00:00, 11.25it/s]


Num heads: 12, head size: 64 =>
Num heads: 6, head size: 64

Num heads: 12, head size: 64 =>
Num heads: 6, head size: 64

Num heads: 12, head size: 64 =>
Num heads: 6, head size: 64

Num heads: 12, head size: 64 =>
Num heads: 6, head size: 64

Num heads: 12, head size: 64 =>
Num heads: 6, head size: 64

Num heads: 12, head size: 64 =>
Num heads: 6, head size: 64

Num heads: 12, head size: 64 =>
Num heads: 6, head size: 64

Num heads: 12, head size: 64 =>
Num heads: 6, head size: 64

Num heads: 12, head size: 64 =>
Num heads: 6, head size: 64

Num heads: 12, head size: 64 =>
Num heads: 6, head size: 64

Num heads: 12, head size: 64 =>
Num heads: 6, head size: 64

Num heads: 12, head size: 64 =>
Num heads: 6, head size: 64

Warning! No positional inputs found for a module, assuming batch size is 1.


INFO:root:Model complexity (After taylor pruning): 88430.840064 MMAC, 85.845504 M params


In [6]:
print('Оригинальная модель--------')
print('общий размер \t {:,}'.format(msize(model.teacher_model)))
print('эмбеддинги \t {:,}'.format(msize(model.teacher_model.embeddings)))
print('энкодер \t {:,}'.format(msize(model.teacher_model.encoder)))
print('Запруненная модель---------')
print('общий размер \t {:,}'.format(msize(model.student_model)))
print('эмбеддинги \t {:,}'.format(msize(model.student_model.embeddings)))
print('энкодер \t {:,}'.format(msize(model.student_model.encoder)))

Оригинальная модель--------
общий размер 	 128,345,088
эмбеддинги 	 42,700,032
энкодер 	 85,054,464
Запруненная модель---------
общий размер 	 85,845,504
эмбеддинги 	 42,700,032
энкодер 	 42,554,880


In [7]:
model.student_model.encoder

BertEncoder(
  (layer): ModuleList(
    (0-11): 12 x BertLayer(
      (attention): BertAttention(
        (self): BertSelfAttention(
          (query): Linear(in_features=768, out_features=384, bias=True)
          (key): Linear(in_features=768, out_features=384, bias=True)
          (value): Linear(in_features=768, out_features=384, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): BertSelfOutput(
          (dense): Linear(in_features=384, out_features=768, bias=True)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (intermediate): BertIntermediate(
        (dense): Linear(in_features=768, out_features=1536, bias=True)
        (intermediate_act_fn): GELUActivation()
      )
      (output): BertOutput(
        (dense): Linear(in_features=1536, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)

In [8]:
model.teacher_model.encoder

BertEncoder(
  (layer): ModuleList(
    (0-11): 12 x BertLayer(
      (attention): BertAttention(
        (self): BertSelfAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Linear(in_features=768, out_features=768, bias=True)
          (value): Linear(in_features=768, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): BertSelfOutput(
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (intermediate): BertIntermediate(
        (dense): Linear(in_features=768, out_features=3072, bias=True)
        (intermediate_act_fn): GELUActivation()
      )
      (output): BertOutput(
        (dense): Linear(in_features=3072, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)